In [5]:
from transformers import Pix2StructProcessor, Pix2StructForConditionalGeneration, pipeline
import requests
from PIL import Image
import json
import accelerate

In [7]:
# Load JSON file
input_file = "/home/g2/ChartQA/ChartQA Dataset/test/test_human.json"  # Replace with your input JSON file path
output_file = "output.json"  # File to save generated answers

with open(input_file, "r") as f:
    data = json.load(f)

In [ ]:
processor = Pix2StructProcessor.from_pretrained('google/deplot')
model = Pix2StructForConditionalGeneration.from_pretrained('google/deplot')

url = "https://raw.githubusercontent.com/vis-nlp/ChartQA/main/ChartQA%20Dataset/val/png/5090.png"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(images=image, text="Generate underlying data table of the figure below:", return_tensors="pt")
predictions = model.generate(**inputs, max_new_tokens=512)
print(processor.decode(predictions[0], skip_special_tokens=True))

In [15]:
# Initialize model
processor = Pix2StructProcessor.from_pretrained('google/deplot')
model = Pix2StructForConditionalGeneration.from_pretrained('google/deplot')

# List to store results
results = []

previous_file_name = ""

# Process each entry in the JSON file
for entry in data:
    file_name = entry["imgname"]
    file_path = "/home/g2/ChartQA/ChartQA Dataset/test/png/{}".format(file_name)
    question = entry["query"]
    expected_answer = entry["label"]

    print(type(question))

    # Reload pipeline if the file name changes
    if file_name != previous_file_name:
        print(f"Loading new image: {file_name}")
        processor = Pix2StructProcessor.from_pretrained('google/deplot')
        model = Pix2StructForConditionalGeneration.from_pretrained('google/deplot')
        previous_file_name = file_name  # Update tracking variable

    # Open the image and verify it's valid
    try:
        image = Image.open(file_path)
        width, height = image.size
        # Validate image dimensions (width and height must not be None)
        if width is None or height is None:
            print(f"Invalid image dimensions for {file_path}")
            continue  # Skip this image if dimensions are invalid
    except (IOError, SyntaxError) as e:
        print(f"Error loading image {file_path}: {e}")
        continue  # Skip this image and move to the next one

    inputs = processor(images=image, text="Using the chart provided, answer the question: {}".format(question), return_tensors="pt")
    predictions = model.generate(**inputs, max_new_tokens=512)
    generated_answer = processor.decode(predictions[0], skip_special_tokens=True)
    print(question)
    print(generated_answer)

    # Store the result
    results.append({
        "imgname": file_name,
        "query": question,
        "label": expected_answer,
        "generated_answer": generated_answer
    })

# Save results to output JSON
with open(output_file, "w") as f:
    json.dump(results, f, indent=4)

print(f"Results saved to {output_file}")

<class 'str'>
Loading new image: 41699051005347.png
How many food item is shown in the bar graph?
Country | Long-term price index in food commodities, 1850-2015, World, 1934 <0x0A> Lamb | 103.7 <0x0A> Corn | 103.13 <0x0A> Barley | 102.46 <0x0A> Rye | 87.37 <0x0A> Beef | 85.27 <0x0A> Wheat | 83.73 <0x0A> Coffee | 82.2 <0x0A> Tea | 68.48 <0x0A> Peanuts | 64.71 <0x0A> Palm oil | 57.6 <0x0A> Pork | 55.36 <0x0A> Rice | 42.48 <0x0A> Sugar | 25.56 <0x0A> Cocoa | 18.81
<class 'str'>


KeyboardInterrupt: 